In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
import math
from datetime import date
import datetime
from functools import reduce
import warnings
warnings.filterwarnings("ignore")

In [99]:
personal_data = pd.read_csv('./data/personal_train.csv')
personal_data = personal_data.drop(['Unnamed: 0'], axis=1)
other_data = pd.read_csv('./data/other_train.csv')
other_data = other_data.drop(['Unnamed: 0'], axis=1)

merged = pd.merge(personal_data,other_data,on=["name","address"])
duplicated = merged.duplicated(subset=["name","address"],keep=False)
duplicates = pd.DataFrame(columns=merged.columns)
indexes_to_drop = []
merged_without_duplicates = pd.DataFrame(columns=merged.columns)
for index, row in merged.iterrows():
    if(duplicated[index]):
        duplicates = duplicates.append(merged.iloc[index],ignore_index=True)
        indexes_to_drop.append(index)

duplicates.sort_values(["name","address"])



merged_without_duplicates = merged

for index in indexes_to_drop:
    merged_without_duplicates = merged_without_duplicates.drop(merged.index[index])

merged = merged_without_duplicates

def checkNaN(arr):
    for item in arr:
        if((type(item) is not np.float64 and type(item) is not float) or not np.isnan(item)):
            return False
    return True

def checkQuestionMark(arr):
    for item in arr:
        if(item is not '?'):
            return False
    return True

def nanOrQuestionMark(arr):
    for item in arr:
        if(item is not '?' and ((type(item) is not np.float64 and type(item) is not float) or not np.isnan(item))):
            return False
    return True

def filter_nan_question_mark(arr):
    result = []
    for item in arr:
        if(item is not '?' and ((type(item) is not np.float64 and type(item) is not float) or not np.isnan(item))):
            result.append(item)
    return result        

def getUniqueRow(startIndex,endIndex):
    row_dict = {}

    for column in duplicates.columns: 
        temp = []
        for index in range(startIndex,endIndex):
            temp.append(duplicates.iloc[index][column])
         
        if(checkNaN(temp)):
            row_dict[column] = np.nan
        elif(checkQuestionMark(temp)):
            row_dict[column] = '?'
        elif(nanOrQuestionMark(temp)):
            row_dict[column] = '?'
        else:
            temp = filter_nan_question_mark(temp)
            value = randint(0,len(temp)-1)
            row_dict[column] = temp[value]
    return row_dict
    
count = 0
lastDuplicate = duplicates.iloc[0]["name"]
columns = duplicates.columns.values
for index,row in duplicates.iterrows():
    entry = duplicates.iloc[index]
    name = entry["name"]
    if(name == lastDuplicate):
        count += 1
    else:
        merged = merged.append(getUniqueRow(index-count,index),ignore_index=True)
        lastDuplicate = name
        count = 1
        
    if(len(duplicates) - 1 == index):
        merged = merged.append(getUniqueRow(index-count+1,index + 1),ignore_index=True)
        
for column in merged.columns:
    merged[column] = merged[column].map(lambda x: np.nan if str(x).strip(" ") == '?' else x)

averageAge = np.nanmean(merged["age"].tolist())

newDate = datetime.datetime(math.floor(datetime.datetime.now().year - averageAge), datetime.datetime.now().month, datetime.datetime.now().day,
                           datetime.datetime.now().hour, datetime.datetime.now().minute).date()

def format_date(string):
    for fmt in ["%Y/%m/%d", "%Y%m%d", "%y-%m-%d", "%Y-%m-%d", "%Y-%m-%d %H:%M:%S", "%Y-%m-%d %H %M %S", "%d/%m/%Y"]:
        try:
            return datetime.datetime.strptime(string, fmt).date()
        except ValueError:
            continue
    raise ValueError(string)

# Funkcia vráti všetky unikátne hodnoty stĺpca
def getUniqueValues(arr):
    return arr.value_counts(normalize=True, dropna=True).index

# Funkcia vráti pomery unikátnych hodnôt stĺpca
def getRatio(arr):
    return arr.value_counts(normalize=True, dropna=True).values

# Nahradí NaN za jednu z hodnôt stĺpca tak, aby pomer hodnôt zostal zachovaný
def fillNanAccordingToRatio(arr):
    return arr.fillna(pd.Series(np.random.choice(getUniqueValues(arr), p=getRatio(arr), size=len(arr))))

#naformatuje datum
merged["date_of_birth"] = merged["date_of_birth"].map(lambda x: format_date(x) if type(x) is str else x)
#doplni chybajuce veky s priemerom
merged["age"] = merged["age"].map(lambda x: math.floor(averageAge) if str(x) == 'nan' or x > 109 else x)
#doplni chybajuce datumy na zaklade priemerneho veku
merged["date_of_birth"] = merged["date_of_birth"].map(lambda x: newDate if str(x) == 'nan' else x)
#opravi datumy ktore su vacsie ako aktualne na zaklade veku

#opravi datumy ktore su vacsie ako aktualne na zaklade veku
i = 0
for date in merged["date_of_birth"]:
    if(date > datetime.datetime.now().date()):
        modifiedDate = datetime.datetime(datetime.datetime.now().year-int(merged["age"][i]), date.month, date.day).date()
        merged["date_of_birth"][i] = modifiedDate 
    i += 1
    
    
    #doplni chybajuce TT4 pomocou algoritmu k najblizsich
k = 5
kNearest = []

i = 0
for tt4 in merged["TT4"]:
    if(str(tt4) == 'nan'):
        kNearest = []
        for index in range(2*k+1):
            if(i+k-index < len(merged) and i+k-index > -1 and i+k-index != i and str(merged["TT4"][i+k-index]) != 'nan' and merged["T4U"][i+k-index] != '?'):
                kNearest.append(merged["TT4"][i+k-index])
        nearestAvg = reduce(lambda x, y: x + y, kNearest) / len(kNearest)
        merged["TT4"][i] = math.floor(nearestAvg)
    i += 1
    
    
#doplni chybajuce T4U pomocou algoritmu k najblizsich
k = 5
kNearest = []

i = 0
for tt4 in merged["T4U"]:
    if(str(tt4) == 'nan'):
        kNearest = []
        for index in range(2*k+1):
            if(i+k-index < len(merged) and i+k-index > -1 and i+k-index != i and str(merged["T4U"][i+k-index]) != 'nan' and merged["T4U"][i+k-index] != '?'):
                kNearest.append(merged["T4U"][i+k-index])
        nearestAvg = reduce(lambda x, y: x + y, kNearest) / len(kNearest)
        merged["T4U"][i] = nearestAvg
    i += 1
    
#doplni chybajuce T4U pomocou algoritmu k najblizsich
k = 5
kNearest = []

i = 0
for tt4 in merged["TSH"]:
    if(str(tt4) == 'nan'):
        kNearest = []
        for index in range(2*k+1):
            if(i+k-index < len(merged) and i+k-index > -1 and i+k-index != i and str(merged["TSH"][i+k-index]) != 'nan' and merged["TSH"][i+k-index] != '?'):
                kNearest.append(merged["TSH"][i+k-index])
        nearestAvg = reduce(lambda x, y: x + y, kNearest) / len(kNearest)
        merged["TSH"][i] = nearestAvg
    i += 1

median = np.nanmedian(merged["T3"].tolist())

merged["T3"] = merged["T3"].map(lambda x: median if str(x) == 'nan' or x == '?' else x)

median = np.nanmedian(merged["education-num"].tolist())

merged["education-num"] = merged["education-num"].map(lambda x: median if str(x) == 'nan' else x)
def mapTrueFalse(x):
    if("t" in str(x).lower()):
        return "t"
    elif("f" in str(x).lower()):
        return "f"
    else:
        return np.nan

merged["sick"] = merged["sick"].map(lambda x: mapTrueFalse(x))
merged["sick"] = fillNanAccordingToRatio(merged["sick"])
    

In [100]:

averageCapital = np.nanmean(merged["capital-loss"].tolist())

merged["capital-loss"] = merged["capital-loss"].map(lambda x: math.floor(averageCapital) if str(x) == 'nan' else x)

merged["sex"] = fillNanAccordingToRatio(merged["sex"])
merged["workclass"] = fillNanAccordingToRatio(merged["workclass"])
merged["native-country"] = fillNanAccordingToRatio(merged["native-country"])
merged["occupation"] = fillNanAccordingToRatio(merged["occupation"])
merged = merged.drop("TBG",axis=1)
merged = merged.drop("TBG measured",axis=1)


In [108]:
merged["FTI"] = merged["FTI"].map(lambda x: float(x))

In [136]:
averageFTI = np.nanmean(merged["FTI"].tolist())
merged["FTI"] = merged["FTI"].map(lambda x: math.floor(averageFTI) if str(x) == 'nan' else x)

for index, row in merged.iterrows():
    newRow = eval(row["medical_info"])
    for key, value in newRow.items():
        merged.loc[index,key] = value
        
merged      
    



,name,address,age,sex,date_of_birth,query hyperthyroid,FTI measured,education,lithium,TT4,...,query hypothyroid,T4U measured,pregnant,thyroid surgery,TSH measured,query on thyroxine,I131 treatment,on thyroxine,T3 measured,psych
0,Edith Boudreaux,"11818 Lori Crossing Apt. 802\nPughstad, DC 78165",78.0,F,1940-02-11,f,t,HS-grad,f,130.0,...,f,t,f,f,t,f,f,f,t,f
1,Terry Terry,"PSC 4657, Box 5446\nAPO AP 58412",75.0,M,1943-04-25,f,t,Some-college,f,93.0,...,f,t,f,f,t,f,f,f,t,f
2,Janet Washington,Unit 9759 Box 9470\nDPO AP 45549,41.0,F,1977-06-09,f,t,10th,f,114.0,...,f,t,f,f,t,f,f,t,t,f
3,Margaret Chabot,"137 Lewis Flat Suite 762\nWest Elizabeth, AL 3...",74.0,F,1944-09-20,f,t,Some-college,f,97.0,...,f,t,f,f,t,f,t,f,f,f
4,Colleen Satterwhite,"995 Frank Stravenue\nSouth Matthewport, TX 81402",37.0,F,1980-11-30,f,t,HS-grad,f,181.0,...,f,t,f,f,t,f,f,t,f,f
5,Judy Smith,"1781 Meredith Skyway Suite 328\nCordovaburgh, ...",51.0,F,1966-11-13,t,f,Assoc-acdm,f,109.0,...,f,f,f,f,f,f,f,f,t,f
6,Stephen Lalk,"2415 Elizabeth Knoll Suite 030\nCordovafort, C...",53.0,M,1965-10-10,f,t,Bachelors,f,74.0,...,f,t,f,f,t,f,f,f,??,f
7,Anna Swartz,"49824 Kim View\nWest Ericborough, VT 42457",27.0,F,1991-04-04,f,f,Assoc-acdm,f,104.0,...,f,f,f,f,t,f,f,f,t,f
8,Betty Kannel,"655 Karen Causeway\nBenjaminberg, NY 46590",77.0,F,1941-08-17,f,t,Bachelors,f,61.0,...,f,t,f,f,t,f,f,f,t,f
9,Patricia Wasilewski,"9197 Juarez Lock\nNew Matthew, VA 12312",60.0,F,1958-01-05,f,t,Assoc-voc,f,128.0,...,f,t,f,f,t,f,f,f,t,f


In [137]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237 entries, 0 to 2236
Data columns (total 44 columns):
name                         2237 non-null object
address                      2237 non-null object
age                          2237 non-null float64
sex                          2237 non-null object
date_of_birth                2237 non-null object
query hyperthyroid           2237 non-null object
FTI measured                 2237 non-null object
education                    2237 non-null object
lithium                      2237 non-null object
TT4                          2237 non-null float64
T4U                          2237 non-null float64
capital-loss                 2237 non-null float64
capital-gain                 2237 non-null float64
tumor                        2237 non-null object
TSH                          2237 non-null float64
T3                           2237 non-null float64
fnlwgt                       2237 non-null float64
hours-per-week               2237 n